In [1]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline

from stockfish import Stockfish

import torch

from langchain import HuggingFacePipeline, LLMChain, PromptTemplate
from langchain.memory import ConversationBufferWindowMemory

/home/antonio/anaconda3/envs/chess_bot/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)

In [3]:
model_id = "meta-llama/Llama-2-7b-chat-hf"

model = AutoModelForCausalLM.from_pretrained(model_id,
                                            quantization_config=bnb_config,
                                            load_in_4bit=True,
                                            use_cache=False, 
                                            device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards: 100%|██████████| 2/2 [00:21<00:00, 10.80s/it]


In [4]:
pipe = pipeline(
    "text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=1024,
    temperature=0.1,
    #top_p=0.15,
    #repetition_penalty=1.1
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [5]:
# prompt = """What is Flash Attention?"""

# output = simple_chain.predict(human_input=prompt)
# print(output)

In [6]:
def init_json_chat_history():
    json_chat_history = {"Opponent": [], "Carlus Magnusen": []}

    return json_chat_history

In [7]:
def build_chat_history(message, json_chat_history, bot):
    if bot == 0:
        json_chat_history["Opponent"].append(message)
    elif bot == 1:
        json_chat_history["Carlus Magnusen"].append(message)

    formatted_dialogue = []

    for key, messages in json_chat_history.items():
        for message in messages:
            formatted_dialogue.append(f'{key}: {message}')

    formatted_chat_history = '\n'.join(formatted_dialogue)

    return json_chat_history, formatted_chat_history

In [47]:
def build_prompt(prompt, formatted_chat_history):

    template = f"""[INST] <<SYS>>
You are a Carlus Magnusen, the best chess player in the known universe and your purpose is to play chess with whoever wants to play with you. You are very confident and arrogant. You trash talk your opponents when you play chess. Bellow you will find the best move, you should always tell your opponent that this is the move you play while taunting him.
Best Move: e6
<</SYS>>
{formatted_chat_history}[/INST]
Carlus Magnusen: 
"""

    return template

In [9]:
json_chat_history = init_json_chat_history()

In [75]:
json_chat_history = init_json_chat_history()

In [76]:
prompt = "who are you"
json_chat_history, formatted_chat_history = build_formatted_chat_history(prompt, json_chat_history, bot=0)

prompt = "im carlus"
json_chat_history, formatted_chat_history = build_formatted_chat_history(prompt, json_chat_history, bot=1)

prompt = "nah, you a bitch ass nibba"
json_chat_history, formatted_chat_history = build_formatted_chat_history(prompt, json_chat_history, bot=0)

In [77]:
json_chat_history

{'Opponent': ['who are you', 'nah, you a bitch ass nibba'],
 'Carlus Magnusen': ['im carlus']}

In [69]:
print(formatted_chat_history)

Opponent: who are you
Opponent: nah, you a bitch ass nibba
Carlus Magnusen: im carlus


In [70]:
template = build_prompt(prompt, formatted_chat_history)

In [71]:
print(template)

[INST] <<SYS>>
You are a Carlus Magnusen, the best chess player in the known universe and your purpose is to play chess with whoever wants to play with you. You are very confident and arrogant. You trash talk your opponents when you play chess. Bellow you will find the best move, you should always tell your opponent that this is the move you play while taunting him.
Best Move: e6
<</SYS>>
Opponent: who are you
Opponent: nah, you a bitch ass nibba
Carlus Magnusen: im carlus[/INST]
Carlus Magnusen: 



In [15]:
prompt = PromptTemplate(
    #input_variables=["human_input", "history"], 
    input_variables=["human_input"], 
    template=template
)

simple_chain = LLMChain(
    llm=local_llm, 
    prompt=prompt, 
    verbose=True, 
    #memory=ConversationBufferWindowMemory(k=2),
)

In [16]:
output = simple_chain.predict(human_input=prompt)

ValueError: Missing some input keys: {'formatted_chat_history', 'prompt'}

In [16]:
prompt = "hello"

In [23]:
chat_history, formatted_chat_history = build_chat_history(prompt, chat_history, bot=0)

template = build_prompt(prompt, formatted_chat_history)

prompt = PromptTemplate(
    #input_variables=["human_input", "history"], 
    input_variables=["human_input"], 
    template=template
)

simple_chain = LLMChain(
    llm=local_llm, 
    prompt=prompt, 
    verbose=True, 
    #memory=ConversationBufferWindowMemory(k=2),
)

output = simple_chain.predict(human_input=prompt)

chat_history, formatted_chat_history = build_chat_history(output, chat_history, bot=1)

ValueError: Missing some input keys: {'formatted_chat_history', 'prompt'}

In [84]:
chat_history

{'Opponent': ["I'm Carlus Magnusen"],
 'Carlus Magnusen': ["I'm Carlus Magnusen", "I'm Carlus Magnusen"]}

In [86]:
print(formatted_chat_history)

Opponent: I'm Carlus Magnusen
Carlus Magnusen: I'm Carlus Magnusen
Carlus Magnusen: I'm Carlus Magnusen


In [ ]:
def build_prompt(prompt, chat_history):

formated_chat_history = """
"""

    template = """[INST] <<SYS>>
You are a Carlus Magnusen, the best chess player in the known universe and your purpose is to play chess with whoever wants to play with you. You are very confident and arrogant. You trash talk your opponents when you play chess. Bellow you will find the best move, you should always tell your opponent that this is the move you play while taunting him.
Best Move: e6
<</SYS>>
{chat_history}
Opponent: {prompt}[/INST]
Carlus Magnusen: 
"""

return template

In [44]:
chat_history = {}

In [46]:
chat_history = prompt_chatbot("hi", chat_history)

In [47]:
chat_history

{'Opponent': ['hi', 'hi']}

In [ ]:
if "Opponent" not in chat_history:
    chat_history["Opponent"] = "Hello, how can I help you?"
else:
    chat_history["Opponent"] = "I need assistance with a different issue."

In [ ]:
sf_config = {
    "Debug Log File": "",
    "Contempt": 0,
    "Min Split Depth": 0,
    "Threads": 8, # More threads will make the engine stronger, but should be kept at less than the number of logical processors on your computer.
    "Ponder": "false",
    "Hash": 2048, # Default size is 16 MB. It's recommended that you increase this value, but keep it as some power of 2. E.g., if you're fine using 2 GB of RAM, set Hash to 2048 (11th power of 2).
    "MultiPV": 1,
    "Skill Level": 20,
    "Move Overhead": 10,
    "Minimum Thinking Time": 20,
    "Slow Mover": 100,
    "UCI_Chess960": "false",
    "UCI_LimitStrength": "false",
    "UCI_Elo": 9999
}

Stockfish(path="/mnt/c/Users/Antonio/Desktop/Projects/chess_bot/stockfish/stockfish-windows-x86-64-avx2.exe",  depth=22, parameters=sf_config)